In [16]:
import ast
import os
import time

import numpy as np
import pandas as pd
from dotenv import load_dotenv
from google import genai
from google.genai import types

load_dotenv()

True

In [3]:
media_df = pd.read_csv("../data/media.csv")
users_df = pd.read_csv("../data/users.csv")

In [3]:
media_df.head()

,id,title,description,genres,tags,isAdult,format,meanScore,popularity,relations,startDate,endDate,season,updatedAt,stats
0,1,"{'native': 'カウボーイビバップ', 'romaji': 'Cowboy Bebo...","Enter a world in the distant future, where Bou...","['Action', 'Adventure', 'Drama', 'Sci-Fi']","[{'id': 63, 'name': 'Space', 'category': 'Sett...",False,TV,86.0,400357,"{'nodes': [{'id': 5, 'title': {'english': ""Cow...",{'year': 1998},{'year': 1999},SPRING,1751484096,"{'scoreDistribution': [{'amount': 549, 'score'..."
1,5,"{'native': 'カウボーイビバップ天国の扉', 'romaji': 'Cowboy ...","As the Cowboy Bebop crew travels the stars, th...","['Action', 'Drama', 'Mystery', 'Sci-Fi']","[{'id': 285, 'name': 'Terrorism', 'category': ...",False,MOVIE,82.0,74052,"{'nodes': [{'id': 1, 'title': {'english': 'Cow...",{'year': 2001},{'year': 2001},SUMMER,1751484161,"{'scoreDistribution': [{'amount': 91, 'score':..."
2,6,"{'native': 'TRIGUN', 'romaji': 'TRIGUN', 'engl...",Vash the Stampede is a wanted man with a habit...,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...","[{'id': 157, 'name': 'Guns', 'category': 'Them...",False,TV,80.0,144709,"{'nodes': [{'id': 30703, 'title': {'english': ...",{'year': 1998},{'year': 1998},SPRING,1751484135,"{'scoreDistribution': [{'amount': 170, 'score'..."
3,7,"{'native': 'Witch Hunter ROBIN', 'romaji': 'Wi...",Robin Sena is a powerful craft user drafted in...,"['Action', 'Drama', 'Mystery', 'Supernatural']","[{'id': 456, 'name': 'Conspiracy', 'category':...",False,TV,69.0,20049,{'nodes': []},{'year': 2002},{'year': 2002},SUMMER,1751476777,"{'scoreDistribution': [{'amount': 42, 'score':..."
4,8,"{'native': '冒険王ビィト', 'romaji': 'Bouken Ou Beet...",It is the dark century and the people are suff...,"['Adventure', 'Fantasy', 'Supernatural']","[{'id': 56, 'name': 'Shounen', 'category': 'De...",False,TV,67.0,2775,"{'nodes': [{'id': 1123, 'title': {'english': '...",{'year': 2004},{'year': 2005},FALL,1751344980,"{'scoreDistribution': [{'amount': 17, 'score':..."


In [4]:
users_df.head()

,id,name,updatedAt,statistics
0,1,Josh,1750794953,"{'anime': {'meanScore': 63.12, 'minutesWatched..."
1,2,matchai,1751214714,"{'anime': {'meanScore': 76.41, 'minutesWatched..."
2,3,Removed,1626987958,"{'anime': {'meanScore': 0, 'minutesWatched': 0..."
3,4,ModChan,1748820691,"{'anime': {'meanScore': 0, 'minutesWatched': 0..."
4,5,kaotikmynd,1746981290,"{'anime': {'meanScore': 81.91, 'minutesWatched..."


In [ ]:
import re

media_df["description"] = media_df["description"].apply(
    lambda x: x.replace("\n", " ").strip() if isinstance(x, str) else x
)
media_df["description"] = media_df["description"].apply(
    lambda x: re.sub(r"<.*?>", "", x) if isinstance(x, str) else x
)

In [6]:
media_df["description"].iloc[420]

"The mysterious island of Houraijima has reappeared after 50 years, and with its reappearance has brought the attack of four gods, the Shitoushin, who have their eyes set on the powers that protect and sustain the island. Now it's up to Inuyasha and his friends, along with Sesshoumaru, to find a way to defeat the powerful Shitoushin.  (Source: Anime News Network) "

In [ ]:
# media_df.to_csv('data/media_cleaned.csv', index=False)s

In [6]:
top_100 = media_df.sort_values(by="popularity", ascending=False).head(100)

In [ ]:
# top_100.to_csv('data/top_100_media.csv', index=False)

In [ ]:
top_100["title"] = top_100["title"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

In [ ]:
def f(row):
    genres = set()
    if row["genres"] is not None:
        genres = set(row["genres"])
    tags = set()
    if row["tags"] is not None:
        tags = set(t["name"] for t in row["tags"])

    unioned_tags = list(genres.union(tags))

    return unioned_tags if unioned_tags else None


media_df["genres"] = media_df["genres"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)
media_df["tags"] = media_df["tags"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)
media_df["tags"] = media_df["tags"].apply(lambda x: None if len(x) == 0 else x)
media_df["genres"] = media_df["genres"].apply(lambda x: None if len(x) == 0 else x)

media_df["genres_tags"] = media_df.apply(f, axis=1)

media_df = media_df[media_df["genres_tags"].notna()]

In [10]:
genres = set()
for tags in media_df["genres_tags"]:
    if tags is not None:
        genres.update(tags)

In [17]:
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

embeddings = {"title": [], "embedding": []}
for genre in genres:
    print(f"Processing genre: {genre}")
    response = client.models.embed_content(
        model="text-embedding-004",
        contents=genre,
    )
    embedding = response.embeddings[0].values

    embeddings["title"].append(genre)
    embeddings["embedding"].append(embedding)

    time.sleep(1)

Processing genre: Virtual World
Processing genre: Feet
Processing genre: Nekomimi
Processing genre: Psychological
Processing genre: Zombie
Processing genre: School
Processing genre: College
Processing genre: Agriculture
Processing genre: Mystery
Processing genre: Tanned Skin
Processing genre: Facial
Processing genre: Judo
Processing genre: Fishing
Processing genre: Camping
Processing genre: Achronological Order
Processing genre: Tragedy
Processing genre: Crime
Processing genre: Body Horror
Processing genre: Sports
Processing genre: Swordplay
Processing genre: Omegaverse
Processing genre: Futanari
Processing genre: Chuunibyou
Processing genre: Gender Bending
Processing genre: Body Image
Processing genre: Femdom
Processing genre: Airsoft
Processing genre: Femboy
Processing genre: Writing
Processing genre: Military
Processing genre: Werewolf
Processing genre: Acrobatics
Processing genre: Fisting
Processing genre: Asexual
Processing genre: Action
Processing genre: Adoption
Processing genre

In [ ]:
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

embeddings = {"title": [], "embedding": []}

top_100.reset_index(drop=True, inplace=True)

for idx, anime in top_100.iterrows():
    desc = anime["description"]
    title = anime["title"]["romaji"]

    response = client.models.embed_content(
        model="text-embedding-004",
        contents=desc,
    )

    print(f"Processing {idx + 1}/{len(top_100)}: {title}")

    embedding = response.embeddings[0].values

    embeddings["title"].append(title)
    embeddings["embedding"].append(embedding)

    time.sleep(1)

In [ ]:
arr = np.vstack(embeddings["embedding"], dtype=np.float16)
np.savetxt("../data/embeddings_genres.csv", arr, delimiter="\t", fmt="%.10f")
pd.Series(embeddings["title"]).to_csv(
    "../data/embeddings_genres_names.csv", index=False, header=False
)